# Retries

Retries are disabled by default. Retries can be enabled with the following example.


In [ ]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License

import json
import logging
import os

from dotenv import load_dotenv
from graphrag_llm.completion import LLMCompletion, create_completion
from graphrag_llm.config import AuthMethod, ModelConfig, RetryConfig, RetryType

load_dotenv()

logging.basicConfig(level=logging.CRITICAL)


api_key = os.getenv("GRAPHRAG_API_KEY")
model_config = ModelConfig(
    model_provider="azure",
    model=os.getenv("GRAPHRAG_MODEL", "gpt-4o"),
    azure_deployment_name=os.getenv("GRAPHRAG_MODEL", "gpt-4o"),
    api_base=os.getenv("GRAPHRAG_API_BASE"),
    api_version=os.getenv("GRAPHRAG_API_VERSION", "2025-04-01-preview"),
    api_key=api_key,
    auth_method=AuthMethod.AzureManagedIdentity if not api_key else AuthMethod.ApiKey,
    retry=RetryConfig(
        type=RetryType.ExponentialBackoff, max_retries=7, base_delay=2.0, jitter=True
    ),
    # Internal option to test error handling and retries
    failure_rate_for_testing=0.5,  # type: ignore
)

llm_completion: LLMCompletion = create_completion(model_config)

response = llm_completion.completion(
    messages="What is the capital of France?",
)

print(f"Metrics for: {llm_completion.metrics_store.id}")
print(json.dumps(llm_completion.metrics_store.get_metrics(), indent=2))

Metrics for: azure/gpt-4o
{
  "attempted_request_count": 1,
  "successful_response_count": 1,
  "failed_response_count": 0,
  "failure_rate": 0.0,
  "requests_with_retries": 1,
  "retries": 2,
  "retry_rate": 0.6666666666666666,
  "compute_duration_seconds": 2.6571085453033447,
  "compute_duration_per_response_seconds": 2.6571085453033447,
  "cache_hit_rate": 0.0,
  "streaming_responses": 0,
  "responses_with_tokens": 1,
  "prompt_tokens": 14,
  "completion_tokens": 8,
  "total_tokens": 22,
  "tokens_per_response": 22.0,
  "responses_with_cost": 1,
  "input_cost": 3.5000000000000004e-05,
  "output_cost": 8e-05,
  "total_cost": 0.000115,
  "cost_per_response": 0.000115
}
